## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
import numpy as np
import os
from random import shuffle
import re

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [3]:
import urllib.request
import zipfile
import lxml.etree

In [4]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [5]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [6]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [7]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [8]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [9]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [10]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [11]:
len(sentences_ted)

266694

In [12]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [46]:
import collections
c=collections.Counter()
for s in sentences_ted:
    for word in s:
        c[word]+=1
#print (c.most_common(1000))
#counts=list(c.most_common(1000))
words_top_ted=list(k[0] for k in c.most_common(1000))
counts_ted_top1000=list(k[1] for k in c.most_common(1000))
#print(counts_ted_top1000[:7])

Plot distribution of top-1000 words

In [30]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [34]:
from gensim.models import Word2Vec

In [37]:
ted_model = Word2Vec(sentences_ted, size=100, window=5, min_count=5, workers=4)

In [38]:
ted_model.save('ted_model')

In [40]:
model_ted=Word2Vec.load('ted_model')

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [41]:
model_ted.most_similar("man")

[('woman', 0.8418140411376953),
 ('guy', 0.8019509315490723),
 ('lady', 0.779926061630249),
 ('girl', 0.7522783279418945),
 ('boy', 0.7340458631515503),
 ('soldier', 0.7012436389923096),
 ('kid', 0.6950050592422485),
 ('gentleman', 0.6791736483573914),
 ('david', 0.6696418523788452),
 ('poet', 0.666323184967041)]

In [42]:
model_ted.most_similar("computer")

[('machine', 0.7558526396751404),
 ('software', 0.7435612678527832),
 ('robot', 0.7120693922042847),
 ('printer', 0.7010622024536133),
 ('3d', 0.6938218474388123),
 ('device', 0.6744407415390015),
 ('program', 0.6506799459457397),
 ('interface', 0.630203366279602),
 ('video', 0.6286095380783081),
 ('camera', 0.6227009296417236)]

In [45]:
model_ted.most_similar('child')
model_ted.most_similar(positive=['child','toy'], negative=['man','woman'])

[('3d', 0.5310326814651489),
 ('robotic', 0.48670458793640137),
 ('personalized', 0.48617684841156006),
 ('motor', 0.4809409976005554),
 ('mobile', 0.47924473881721497),
 ('synthetic', 0.47842317819595337),
 ('circuit', 0.4707862138748169),
 ('neural', 0.4674309194087982),
 ('memory', 0.46332329511642456),
 ('chemical', 0.46291378140449524)]

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [47]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

In [48]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [49]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [50]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [51]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [52]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [53]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

4267112
853422


In [82]:
sentences_wiki_2=[]
for k in sentences_wiki:
    sentences_wiki_2.append(k.split())
print(sentences_wiki_2[0])

['analysis', 'of', 'the', 'new', 'towns', 'in', 'wales', 'and', 'england', 'of', 'the', 'period', 'has', 'identified', 'common', 'similarities', 'in', 'their', 'design', 'with', 'most', 'featuring', 'a', 't', 'shaped', 'street', 'plan', 'stretching', 'away', 'from', 'the', 'coast', 'or', 'river', 'aligned', 'with', 'a', 'castle', 'that', 'is', 'usually', 'positioned', 'at', 'a', 'top', 'corner', 'of', 'the', 't']


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [71]:
wc=collections.Counter()
for s in sentences_wiki:
    for word in s.split():
        wc[word]+=1
words_top_wiki=list(k[0] for k in wc.most_common(1000))
counts_wiki_top1000=list(k[1] for k in wc.most_common(1000))
print(words_top_wiki[:7])

[1284867, 546435, 500005, 434119, 399090, 357057, 214959]


In [89]:
print(words_top_wiki[:7])

['the', 'of', 'and', 'in', 'to', 'a', 'was']


Plot distribution of top-1000 words

In [73]:
hist, edges = np.histogram(counts_wiki_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

In [83]:
wiki_model = Word2Vec(sentences_wiki_2, size=100, window=5, min_count=5, workers=4)

In [84]:
wiki_model.save('wiki_model')

In [85]:
model_wiki=Word2Vec.load('wiki_model')

In [86]:
model_wiki.most_similar("man")

[('woman', 0.711510956287384),
 ('boy', 0.6445591449737549),
 ('girl', 0.6333346366882324),
 ('person', 0.6003431677818298),
 ('dog', 0.5928398370742798),
 ('soldier', 0.5709395408630371),
 ('creature', 0.5626027584075928),
 ('mask', 0.5576151609420776),
 ('joke', 0.5341308116912842),
 ('kid', 0.5297766327857971)]

In [87]:
model_wiki.most_similar("computer")

[('software', 0.7895188331604004),
 ('computers', 0.7731009721755981),
 ('hardware', 0.7636091113090515),
 ('user', 0.7558916807174683),
 ('technology', 0.7288669943809509),
 ('simulation', 0.7087910771369934),
 ('functionality', 0.702410101890564),
 ('virtual', 0.6961725950241089),
 ('handheld', 0.6957165598869324),
 ('graphics', 0.6949478387832642)]

In [88]:
model_ted.most_similar('child')
model_ted.most_similar(positive=['child','toy'], negative=['man','woman'])

[('3d', 0.5310326814651489),
 ('robotic', 0.48670458793640137),
 ('personalized', 0.48617684841156006),
 ('motor', 0.4809409976005554),
 ('mobile', 0.47924473881721497),
 ('synthetic', 0.47842317819595337),
 ('circuit', 0.4707862138748169),
 ('neural', 0.4674309194087982),
 ('memory', 0.46332329511642456),
 ('chemical', 0.46291378140449524)]

#### t-SNE visualization

In [90]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [91]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)